In [1]:
from datetime import datetime
from airflow import DAG
from datetime import datetime, timedelta
from airflow.operators.empty import EmptyOperator
from airflow.operators.python import PythonOperator, BranchPythonOperator

import os
import pandas as pd
import FinanceDataReader as fdr
from datetime import datetime
from dateutil.relativedelta import relativedelta

import exchange_calendars as ecals

from dags.model import rasterize, inference

In [5]:
def predict(date):
    ks_date = datetime.strptime(date,'%Y-%m-%d')
    ks_date = ks_date + relativedelta(days=1)
    ks_date = ks_date.strftime("%Y-%m-%d")

    # 주식 코드 db 테이블에서 반복문으로 각 코드 가져오기
    file_name = f'./dags/database/symbol_data/korea_stock_symbols_2024-03-08.csv'
    stock_table = pd.read_csv(file_name)

    column = ['Date', 'Name', 'Code', 'Close_Price', \
                'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7']
    predict_df = pd.DataFrame(columns=column)


    for _, stock in stock_table.iterrows():
        
        code = stock['value'].split(':')[1]
        name = stock['label']

        close_price = fdr.DataReader(code, ks_date, ks_date).iloc[0]['Close']
        try:
            stock_image = rasterize(code, ks_date)
        except:
            print(code, ks_date)
            close_price = fdr.DataReader(code, ks_date, ks_date).iloc[0]['Close']
            
        preds = inference(stock_image)
        preds_result = []
        for idx, pct in preds:
            preds_result.append(str(idx)+'/'+str(round(pct,2)))
        
        result = [ks_date, name, code, close_price]
        result.extend(preds_result)

        predict_df = pd.concat([predict_df, pd.DataFrame([result],columns=column)])

    save_dir = './dags/database/predict_table'
    predict_df.to_csv(os.path.join(save_dir, f"predict_{ks_date}.csv"))

In [6]:
predict("2024-03-12")

/tmp/ipykernel_418572/1191070724.py:35 FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

278470 2024-03-13
452430 2024-03-13
443670 2024-03-13
199430 2024-03-13
418620 2024-03-13
360350 2024-03-13
068100 2024-03-13
473050 2024-03-13
469900 2024-03-13
468760 2024-03-13
472230 2024-03-13
473370 2024-03-13


In [ ]:
278470 2024-03-13
452430 2024-03-13
443670 2024-03-13
199430 2024-03-13
418620 2024-03-13
360350 2024-03-13
068100 2024-03-13
473050 2024-03-13
469900 2024-03-13
468760 2024-03-13
472230 2024-03-13
473370 2024-03-13

In [7]:
folder_name = 'symbol_data'
file_name = 'korea_stock_symbols.csv'
file_path = os.path.join(folder_name, file_name)

# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

In [5]:
import pandas as pd
kospi_list = fdr.StockListing('KOSPI')
kosdaq_list = fdr.StockListing('KOSDAQ')
korea_stock_list = pd.concat([kospi_list, kosdaq_list])
korea_stock_symbol = korea_stock_list[['Code', 'Name']]

# columns = ['Code']
# korea_stock_symbol['Code'] = 'KRX:' + korea_stock_symbol['Code']

In [1]:
from tqdm import tqdm
import FinanceDataReader as fdr

In [6]:
code_list, name_list = [], []

In [17]:
df = fdr.DataReader(code,"2024-03-08").index
df

DatetimeIndex(['2024-03-08', '2024-03-11', '2024-03-12', '2024-03-13'], dtype='datetime64[ns]', name='Date', freq=None)

In [12]:
kospi_list = fdr.StockListing('KOSPI')
kosdaq_list = fdr.StockListing('KOSDAQ')
korea_stock_list = pd.concat([kospi_list, kosdaq_list])
korea_stock_symbol = korea_stock_list[['Code', 'Name']]

code_list, name_list = [], []
for _, stock in korea_stock_symbol.iterrows():
    code = stock['Code']
    name = stock['Name']
    if len(fdr.DataReader(code,"2024-03-08")) < 40:
        continue
    code_list.append(code)
    name_list.append(name)

korea_stock_df = pd.DataFrame({'Code': code_list, 'Name' : name_list})
korea_stock_df['Code'] = 'KRX:' + korea_stock_df['Code']

KeyboardInterrupt: 

In [10]:
korea_stock_df

,Code,Name
0,005930,삼성전자
1,000660,SK하이닉스
2,373220,LG에너지솔루션
3,207940,삼성바이오로직스
4,005380,현대차
...,...,...
2642,161570,THE MIDONG
2643,413600,키움제6호스팩
2644,438580,엔에이치스팩25호
2645,032685,소프트센우


In [11]:
korea_stock_df['Code'] = 'KRX:' + korea_stock_df['Code']
korea_stock_df

,Code,Name
0,KRX:005930,삼성전자
1,KRX:000660,SK하이닉스
2,KRX:373220,LG에너지솔루션
3,KRX:207940,삼성바이오로직스
4,KRX:005380,현대차
...,...,...
2642,KRX:161570,THE MIDONG
2643,KRX:413600,키움제6호스팩
2644,KRX:438580,엔에이치스팩25호
2645,KRX:032685,소프트센우


In [9]:
fdr.DataReader('472230')

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2024-03-04,4000,5700,2100,2100,89618041,NaN
2024-03-05,2105,2110,2065,2080,1844735,-0.009524
2024-03-06,2075,2080,2065,2075,332416,-0.002404
2024-03-07,2070,2090,2070,2080,255261,0.002410
2024-03-08,2095,2105,2085,2090,110000,0.004808
2024-03-11,2090,2100,2085,2090,146048,0.000000
2024-03-12,2085,2100,2085,2095,67992,0.002392
2024-03-13,2095,2095,2085,2095,64649,0.000000


In [35]:
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.datetime.now()
bef = today - relativedelta(months=3)
bef_str = bef.strftime("%Y-%m-%d")
today_str = today.strftime("%Y-%m-%d")

In [36]:
print(bef_str, today_str)

2023-12-13 2024-03-13


In [40]:
stocks = fdr.StockListing('KOSPI')[['Code','Name']]
for code in stocks['Code']:
    df = fdr.DataReader(code,bef_str, today_str)
    break
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-12-13,73300,73500,72800,72800,13116766,-0.009524
2023-12-14,74100,74300,72500,73100,27567593,0.004121
2023-12-15,73800,74000,73200,73300,15419815,0.002736
2023-12-18,73300,73400,72800,72900,9690551,-0.005457
2023-12-19,73000,73400,72800,73400,8907632,0.006859
2023-12-20,74200,74900,73800,74800,16870156,0.019074
2023-12-21,74600,75000,74300,75000,13478766,0.002674
2023-12-22,75800,76300,75400,75900,14515608,0.012000
2023-12-26,76100,76700,75700,76600,13164909,0.009223


In [8]:
kospi_list = fdr.StockListing('KOSPI')
kosdaq_list = fdr.StockListing('KOSDAQ')

korea_stock_list = pd.concat([kospi_list, kosdaq_list])

korea_stock_symbol = korea_stock_list[['Code', 'Name']]

korea_stock_symbol['Code'] = 'KRX:' + korea_stock_symbol['Code']

print(korea_stock_symbol.head())



         Code      Name
0  KRX:005930      삼성전자
1  KRX:000660    SK하이닉스
2  KRX:373220  LG에너지솔루션
3  KRX:207940  삼성바이오로직스
4  KRX:005935     삼성전자우


/tmp/ipykernel_123780/516760139.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  korea_stock_symbol['Code'] = 'KRX:' + korea_stock_symbol['Code']


In [27]:
korea_stock_symbol.rename(columns={'Name': 'label', 'Code': 'value'}, inplace=True)
korea_stock_symbol.to_csv(file_path, index=False, encoding='utf-8-sig')

print(f'CSV 파일이 {file_path}에 저장되었습니다.')

CSV 파일이 symbol_data/korea_stock_symbols.csv에 저장되었습니다.


/var/folders/b3/x6l_dngd7mxg34spbn8446pm0000gn/T/ipykernel_5337/1372236534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  korea_stock_symbol.rename(columns={'Name': 'label', 'Code': 'value'}, inplace=True)
